In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd  # asi se suele importar Pandas
palette = 'colorblind'
sns.set_palette(palette); sns.set_color_codes(palette)  # Fija los nombres cortos para los colores según la paleta de seaborn

# Manipulación de datos y Pandas

**Pandas** es un paquete que Python que provee estructuras de datos rápidas, flexibles y expresivas diseñadas para trabajar con _rotulados_ y/o _relacionales_. Conceptualmente se pueden pensar como _arrays_ de NumPy donde las filas y colummnas están rótuladas. O de forma similar como una _planilla de cálculo_ bajo Python.

Asi como NumPy es una muy buena herramienta para trabajar con números, vectores, algebra lineal, etc. Pandas es adecuado para trabajar con:

* Datos tabulares y heterogeneos (flotantes, string, enteros, etc)
* Series temporales
* Los mismos datos que se pueden manipular con _arreglos_ de NumPy!

Key features:
    
- Easy handling of **missing data**
- **Size mutability**: columns can be inserted and deleted from DataFrame and higher dimensional objects
- Automatic and explicit **data alignment**: objects can be explicitly aligned to a set of labels, or the data can be aligned automatically
- Powerful, flexible **group by functionality** to perform split-apply-combine operations on data sets
- Intelligent label-based **slicing, fancy indexing, and subsetting** of large data sets
- Intuitive **merging and joining** data sets
- Flexible **reshaping and pivoting** of data sets
- **Hierarchical labeling** of axes
- Robust **IO tools** for loading data from flat files, Excel files, databases, and HDF5
- **Time series functionality**: date range generation and frequency conversion, moving window statistics, moving window linear regressions, date shifting and lagging, etc.


Esta notebook están basada en las [notebooks](https://github.com/fonnesbeck/Bios8366/tree/master/notebooks) de Chris Fonnesbeck sobre Pandas.

Por que es importante tener una herramienta como Pandas?

<img src="imagenes/analisis.jpeg"  width=300>

Pandas introduce fundamentalmente 3 nuevas estructuras de datos. La `Series` el `DataFrame`, y el `Index`. Empezemos por la primera de estas.

### Series

Una _Series_ de Pandas es un conjunto unidimensional de datos (similar a un _array_) acompañados de un índice que "rotula" a cada elemento del vector. Puede ser creada a partir de un array o tupla o lista.

In [2]:
conteo = pd.Series([632, 1638, 569, 115])
conteo

0     632
1    1638
2     569
3     115
dtype: int64

La primer columna es el índice y la segunda nuestros datos. Como omitimos indicar un índice Pandas asignó automáticamente una secuencia de enteros (empezando por 0). 

A partir de una serie es posible obtener solo el _array_ de NumPy "contenido" en ella.

In [3]:
conteo.values

array([ 632, 1638,  569,  115])

Como también es posible obtener solo el índice.

In [4]:
conteo.index

RangeIndex(start=0, stop=4, step=1)

Es importante notar que los arreglos de NumPy también tienen índices, solo que estos están implícitios y siemrpe son enteros comenzando desde el 0. En cambio los índices en Pandas son explícitos. Podemos asignar rotulos que tengan sentido según nuestros datos. Si nuestros datos representan la cantidad de diversas bacterias en una muestra, podríamos tener algo como:

In [5]:
bacteria = pd.Series([632, 1638, 569, 115], 
    index=['Firmicutes', 'Proteobacteria', 'Actinobacteria', 'Bacteroidetes'])

bacteria

Firmicutes         632
Proteobacteria    1638
Actinobacteria     569
Bacteroidetes      115
dtype: int64

Ahora que tenemos  rótulos por un lado y datos por el otro da la impresión que una serie se podría pensar también como una especie de diccionario! De hecho podemos usar los rótulos para referirnos directamente a valores en la serie.

In [6]:
bacteria['Actinobacteria']

569

O usando una sintaxis ligeramente más simple

In [7]:
bacteria.Actinobacteria

569

A diferencia de los diccionarios es posible hacer `slicing`

In [8]:
bacteria['Proteobacteria':]

Proteobacteria    1638
Actinobacteria     569
Bacteroidetes      115
dtype: int64

Al igual que con los arreglos de NumPy podemos usar booleanos para indexar una serie. Si quisieramos el conteo de  bacterias para todas aquellas cuyos nombres terminan en "bacteria" podríamos hacer:

In [9]:
bacteria[[name.endswith('bacteria') for name in bacteria.index]]

Proteobacteria    1638
Actinobacteria     569
dtype: int64

U operaciones como:

In [10]:
bacteria[bacteria>1000]

Proteobacteria    1638
dtype: int64

También es posible hacer uso del índice _implicito_ (como en listas y arreglos)

In [11]:
bacteria[1]

1638

We can give both the array of values and the index meaningful labels themselves:

In [12]:
bacteria.name = 'counts'
bacteria.index.name = 'phylum'
bacteria

phylum
Firmicutes         632
Proteobacteria    1638
Actinobacteria     569
Bacteroidetes      115
Name: counts, dtype: int64

Es posible operarcon los valores de una serie manteniendo los rótulos sin modificar

In [13]:
np.log(bacteria)

phylum
Firmicutes        6.448889
Proteobacteria    7.401231
Actinobacteria    6.343880
Bacteroidetes     4.744932
Name: counts, dtype: float64

Como ya dijimos es posible pensar en una `Series` como si fuera una especie de diccionario, incluso podemos crear series a partir de diccionarios!

In [14]:
bacteria_dict = {'Firmicutes': 632, 'Proteobacteria': 1638, 'Actinobacteria': 569, 'Bacteroidetes': 115}
pd.Series(bacteria_dict)

Actinobacteria     569
Bacteroidetes      115
Firmicutes         632
Proteobacteria    1638
dtype: int64

Si observan con atención verán que Pandas ordenó alfabeticamente los indices en la `Series` (junto con sus valores asociados).

Si queremos añgun orden en particular podemos especificar los índices. Incluso podemos pasar rotulos para valores que no existen. En ese caso Pandas interpretará que tenemos datos faltantes (_missing data_) y lo indicará usando un tipo especial de _float_ `NaN` (not a number).

In [15]:
bacteria2 = pd.Series(bacteria_dict, index=['Cyanobacteria','Firmicutes','Proteobacteria','Actinobacteria'])
bacteria2

Cyanobacteria        NaN
Firmicutes         632.0
Proteobacteria    1638.0
Actinobacteria     569.0
dtype: float64

Es posible preguntarle a Pandas si tenemos valores faltantes

In [16]:
bacteria2.isnull()

Cyanobacteria      True
Firmicutes        False
Proteobacteria    False
Actinobacteria    False
dtype: bool

Los índices no son solo una conveniencia para manipular datos haciendo referencia a nombres que nos puede resultar más familiares o convenientes (comparado con recordar la posición de los datos). Los índices son usados para **alienar datos** al operar con más de una serie, por ej podríamos queres obtner el total de bacterias en dos conjuntos de datos.

In [17]:
bacteria

phylum
Firmicutes         632
Proteobacteria    1638
Actinobacteria     569
Bacteroidetes      115
Name: counts, dtype: int64

In [18]:
bacteria + bacteria2

Actinobacteria    1138.0
Bacteroidetes        NaN
Cyanobacteria        NaN
Firmicutes        1264.0
Proteobacteria    3276.0
dtype: float64

Como verán Pandas sumó solo los valores para  los cuales los índices de ambas `Series` coinciden, propagando además los valores faltantes (`NaN`). 

¿Cómo se compara esto con sumar dos arreglos de NumPy de distinta longitud?

## DataFrame

Al analizar datos es común que tengamos que trabajar con datos multivariados. Es decir con más de uan variable. Para esos casos sería útil tener algo como una `Series` donde para cada índice tengamos más de una columna de valores. Ese objeto se llama `DataFrame`.

Un `DataFrame` es una estructura de datos tabular que se puede pensar como una colección de `Series` (que comparten un mismo índice). También es posible pensar un `DataFrame` como una generalización de un arreglo de NumPy o de una diccionario. 

In [19]:
datos = pd.DataFrame({'conteo':[632, 1638, 569, 115, 433, 1130, 754, 555],
                     'paciente':[1, 1, 1, 1, 2, 2, 2, 2],
                     'phylum':['Firmicutes', 'Proteobacteria', 'Actinobacteria', 
    'Bacteroidetes', 'Firmicutes', 'Proteobacteria', 'Actinobacteria', 'Bacteroidetes']})
datos

,conteo,paciente,phylum
0,632,1,Firmicutes
1,1638,1,Proteobacteria
2,569,1,Actinobacteria
3,115,1,Bacteroidetes
4,433,2,Firmicutes
5,1130,2,Proteobacteria
6,754,2,Actinobacteria
7,555,2,Bacteroidetes


Lo primero que notamos es que `Jupyter` le pone un poco de estilo al `DataFrame`, y lo muestra como una tabla con algunas mejoras estéticas (comaprado con un montón de números como se ven arreglos, listas y series).

También podemos ver que contrario a un arreglo de NumPy en un `DataFrame` podemos tener datos de distinto tipo (enteros y _strings_ en este caso). Además se ve que las columnas están ordenadas alfabeticamente, podemos cambiar el orden indexando el `DataFrame` en el orden preferido.

In [20]:
datos[['paciente', 'phylum', 'conteo']]

,paciente,phylum,conteo
0,1,Firmicutes,632
1,1,Proteobacteria,1638
2,1,Actinobacteria,569
3,1,Bacteroidetes,115
4,2,Firmicutes,433
5,2,Proteobacteria,1130
6,2,Actinobacteria,754
7,2,Bacteroidetes,555


Los `DataFrame` tienen dos `index`, el mismo que ya vimos para las series que se corresponden con las filas y uno nuevo que se corresponde con las columnas

In [21]:
datos.columns

Index(['conteo', 'paciente', 'phylum'], dtype='object')

Es posible acceder a los valores de las columnas como con un diccionario

In [22]:
datos['conteo']

0     632
1    1638
2     569
3     115
4     433
5    1130
6     754
7     555
Name: conteo, dtype: int64

o por atributo (un atributo es el nombre que se le da a un dato o propiedad en programación orientada a objetos)

In [23]:
datos.conteo

0     632
1    1638
2     569
3     115
4     433
5    1130
6     754
7     555
Name: conteo, dtype: int64

Una posible fuente  de confusión es que la sintaxis que acabamos de ver devuelve filas en una `Series`, pero columnas en un `DataFrame`. Si queremos acceder a las filas de un `DataFrame` podemos hacerlo usando el atributo `ix`:

In [24]:
datos.ix[3]

conteo                115
paciente                1
phylum      Bacteroidetes
Name: 3, dtype: object

¿Que pasa si intentamos acceder a una fila usando la sintaxis `datos[3]`?

La serie que se obtieen al indexar un `DataFrame` es una _vista_ (_view_) del `DataFrame` y NO una copia. Por lo que hay que tener cuidado al manipularla, por ello Pandas nos devuelve una advertencia.

In [25]:
cont = datos.conteo
cont

0     632
1    1638
2     569
3     115
4     433
5    1130
6     754
7     555
Name: conteo, dtype: int64

In [26]:
cont[5] = 0
cont

/home/osvaldo/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


0     632
1    1638
2     569
3     115
4     433
5       0
6     754
7     555
Name: conteo, dtype: int64

In [27]:
datos

,conteo,paciente,phylum
0,632,1,Firmicutes
1,1638,1,Proteobacteria
2,569,1,Actinobacteria
3,115,1,Bacteroidetes
4,433,2,Firmicutes
5,0,2,Proteobacteria
6,754,2,Actinobacteria
7,555,2,Bacteroidetes


Si queremos moficiar una `Series` que proviene de un `DataFrame` puede ser buena idea hacer una copia primero.

In [28]:
cont = datos.conteo.copy()
cont[5] = 1000
datos

,conteo,paciente,phylum
0,632,1,Firmicutes
1,1638,1,Proteobacteria
2,569,1,Actinobacteria
3,115,1,Bacteroidetes
4,433,2,Firmicutes
5,0,2,Proteobacteria
6,754,2,Actinobacteria
7,555,2,Bacteroidetes


Es posible crear columnas usando asignaciones

In [29]:
datos['year'] = 2013
datos

,conteo,paciente,phylum,year
0,632,1,Firmicutes,2013
1,1638,1,Proteobacteria,2013
2,569,1,Actinobacteria,2013
3,115,1,Bacteroidetes,2013
4,433,2,Firmicutes,2013
5,0,2,Proteobacteria,2013
6,754,2,Actinobacteria,2013
7,555,2,Bacteroidetes,2013


Pero para hacer esto no es posible usar la sintaxiis de  atributo

In [30]:
datos.tratamiento = 1

Aunque esto crea el atributo y le asgina el valor 1

In [31]:
datos.tratamiento

1

Este cambio no se ve reflejado en el `DataFrame`

In [32]:
datos

,conteo,paciente,phylum,year
0,632,1,Firmicutes,2013
1,1638,1,Proteobacteria,2013
2,569,1,Actinobacteria,2013
3,115,1,Bacteroidetes,2013
4,433,2,Firmicutes,2013
5,0,2,Proteobacteria,2013
6,754,2,Actinobacteria,2013
7,555,2,Bacteroidetes,2013


Podemos agregar una `Series` como una nueva columna en un `DataFrame`, el resultado dependerá de los índices de ambos objetos.

In [33]:
tratamiento = pd.Series([0]*4 + [1]*2)
tratamiento

0    0
1    0
2    0
3    0
4    1
5    1
dtype: int64

In [34]:
datos['tratamiento'] = tratamiento
datos

,conteo,paciente,phylum,year,tratamiento
0,632,1,Firmicutes,2013,0.0
1,1638,1,Proteobacteria,2013,0.0
2,569,1,Actinobacteria,2013,0.0
3,115,1,Bacteroidetes,2013,0.0
4,433,2,Firmicutes,2013,1.0
5,0,2,Proteobacteria,2013,1.0
6,754,2,Actinobacteria,2013,NaN
7,555,2,Bacteroidetes,2013,NaN


¿Que pasa si intentamos agregar una nueva columna que no sea una serie y cuyo longitud no coincida con la del `DataFrame`?

In [35]:
datos['mes'] = ['enero'] * len(datos)
datos

,conteo,paciente,phylum,year,tratamiento,mes
0,632,1,Firmicutes,2013,0.0,enero
1,1638,1,Proteobacteria,2013,0.0,enero
2,569,1,Actinobacteria,2013,0.0,enero
3,115,1,Bacteroidetes,2013,0.0,enero
4,433,2,Firmicutes,2013,1.0,enero
5,0,2,Proteobacteria,2013,1.0,enero
6,754,2,Actinobacteria,2013,NaN,enero
7,555,2,Bacteroidetes,2013,NaN,enero


Podemos usar `del` para eliminar columnas de la misma forma que en diccionarios.

In [36]:
del datos['mes']
datos

,conteo,paciente,phylum,year,tratamiento
0,632,1,Firmicutes,2013,0.0
1,1638,1,Proteobacteria,2013,0.0
2,569,1,Actinobacteria,2013,0.0
3,115,1,Bacteroidetes,2013,0.0
4,433,2,Firmicutes,2013,1.0
5,0,2,Proteobacteria,2013,1.0
6,754,2,Actinobacteria,2013,NaN
7,555,2,Bacteroidetes,2013,NaN


Es posible _extaer_ los datos de un `DataFrame` en forma de arreglo de NumPy.

In [37]:
datos.values

array([[632, 1, 'Firmicutes', 2013, 0.0],
       [1638, 1, 'Proteobacteria', 2013, 0.0],
       [569, 1, 'Actinobacteria', 2013, 0.0],
       [115, 1, 'Bacteroidetes', 2013, 0.0],
       [433, 2, 'Firmicutes', 2013, 1.0],
       [0, 2, 'Proteobacteria', 2013, 1.0],
       [754, 2, 'Actinobacteria', 2013, nan],
       [555, 2, 'Bacteroidetes', 2013, nan]], dtype=object)

Fijense que el `dtype` del arreglo es `object`. Esto se debe a la mezcla de enteros _strings_ y flotantes (`Nan`). El `dtype` es elegido por Pandas automaticamente de forma tal de acomodar todos los tipos de valores presentes en el `DataFrame`.

## Index

La última estructura de datos que nos queda ver es `Index`, la cual ya la hemos estado usando durante todo el capítulo. Solo que ahora hablaremos de ella de forma un poco más explícita.

In [38]:
datos.index

RangeIndex(start=0, stop=8, step=1)

Los `Index` son inmutables

In [39]:
#datos.index[0] = 15

Esto está pensado para permitir que los `Index` se compartan entre objetos sin riesgo de que se modifiquen en algún momento.

In [40]:
bacteria2.index = bacteria.index

In [41]:
bacteria2

phylum
Firmicutes           NaN
Proteobacteria     632.0
Actinobacteria    1638.0
Bacteroidetes      569.0
dtype: float64

## Importing data

A key, but often under-appreciated, step in data analysis is importing the data that we wish to analyze. Though it is easy to load basic data structures into Python using built-in tools or those provided by packages like NumPy, it is non-trivial to import structured data well, and to easily convert this input into a robust data structure:

    genes = np.loadtxt("genes.csv", delimiter=",", dtype=[('gene', '|S10'), ('value', '<f4')])

Pandas provides a convenient set of functions for importing tabular data in a number of formats directly into a `DataFrame` object. These functions include a slew of options to perform type inference, indexing, parsing, iterating and cleaning automatically as data are imported.

Let's start with some more bacteria data, stored in csv format.

In [ ]:
!cat ../data/microbiome.csv

This table can be read into a DataFrame using `read_csv`:

In [ ]:
mb = pd.read_csv("../data/microbiome.csv")
mb

Notice that `read_csv` automatically considered the first row in the file to be a header row.

We can override default behavior by customizing some the arguments, like `header`, `names` or `index_col`.

In [ ]:
pd.read_csv("../data/microbiome.csv", header=None).head()

`read_csv` is just a convenience function for `read_table`, since csv is such a common format:

In [ ]:
mb = pd.read_table("../data/microbiome.csv", sep=',')

The `sep` argument can be customized as needed to accomodate arbitrary separators. For example, we can use a regular expression to define a variable amount of whitespace, which is unfortunately very common in some data formats: 
    
    sep='\s+'

For a more useful index, we can specify the first two columns, which together provide a unique index to the data.

In [ ]:
mb = pd.read_csv("../data/microbiome.csv", index_col=['Patient','Taxon'])
mb.head()

This is called a *hierarchical* index, which we will revisit later in the section.

If we have sections of data that we do not wish to import (for example, known bad data), we can populate the `skiprows` argument:

In [ ]:
pd.read_csv("../data/microbiome.csv", skiprows=[3,4,6]).head()

If we only want to import a small number of rows from, say, a very large data file we can use `nrows`:

In [ ]:
pd.read_csv("../data/microbiome.csv", nrows=4)

Alternately, if we want to process our data in reasonable chunks, the `chunksize` argument will return an iterable object that can be employed in a data processing loop. For example, our microbiome data are organized by bacterial phylum, with 15 patients represented in each:

In [ ]:
pd.read_csv("../data/microbiome.csv", chunksize=15)

In [ ]:
data_chunks = pd.read_csv("../data/microbiome.csv", chunksize=15)

mean_tissue = pd.Series({chunk.Taxon[0]: chunk.Tissue.mean() for chunk in data_chunks})
    
mean_tissue

Most real-world data is incomplete, with values missing due to incomplete observation, data entry or transcription error, or other reasons. Pandas will automatically recognize and parse common missing data indicators, including `NA` and `NULL`.

In [ ]:
!cat ../data/microbiome_missing.csv

In [ ]:
pd.read_csv("../data/microbiome_missing.csv").head(20)

Above, Pandas recognized `NA` and an empty field as missing data.

In [ ]:
pd.isnull(pd.read_csv("../data/microbiome_missing.csv")).head(20)

Unfortunately, there will sometimes be inconsistency with the conventions for missing data. In this example, there is a question mark "?" and a large negative number where there should have been a positive integer. We can specify additional symbols with the `na_values` argument:
   

In [ ]:
pd.read_csv("../data/microbiome_missing.csv", na_values=['?', -99999]).head(20)

These can be specified on a column-wise basis using an appropriate dict as the argument for `na_values`.

### Microsoft Excel

Since so much financial and scientific data ends up in Excel spreadsheets (regrettably), Pandas' ability to directly import Excel spreadsheets is valuable. This support is contingent on having one or two dependencies (depending on what version of Excel file is being imported) installed: `xlrd` and `openpyxl` (these may be installed with either `pip` or `easy_install`).

The read_excel convenience function in pandas imports a specific sheet from an Excel file

In [ ]:
mb = pd.read_excel('../data/microbiome/MID2.xls', sheetname='Sheet 1', header=None)
mb.head()

There are several other data formats that can be imported into Python and converted into DataFrames, with the help of buitl-in or third-party libraries. These include JSON, XML, HDF5, relational and non-relational databases, and various web APIs. These are beyond the scope of this tutorial, but are covered in [Python for Data Analysis](http://shop.oreilly.com/product/0636920023784.do).

## Pandas Fundamentals

This section introduces the new user to the key functionality of Pandas that is required to use the software effectively.

For some variety, we will leave our digestive tract bacteria behind and employ some baseball data.

In [ ]:
baseball = pd.read_csv("../data/baseball.csv", index_col='id')
baseball.head()

Notice that we specified the `id` column as the index, since it appears to be a unique identifier. We could try to create a unique index ourselves by combining `player` and `year`:

In [ ]:
player_id = baseball.player + baseball.year.astype(str)
baseball_newind = baseball.copy()
baseball_newind.index = player_id
baseball_newind.head()

This looks okay, but let's check:

In [ ]:
baseball_newind.index.is_unique

So, indices need not be unique. Our choice is not unique because some players change teams within years.

In [ ]:
pd.Series(baseball_newind.index).value_counts()

The most important consequence of a non-unique index is that indexing by label will return multiple values for some labels:

In [ ]:
baseball_newind.loc['wickmbo012007']

We will learn more about indexing below.

We can create a truly unique index by combining `player`, `team` and `year`:

In [ ]:
player_unique = baseball.player + baseball.team + baseball.year.astype(str)
baseball_newind = baseball.copy()
baseball_newind.index = player_unique
baseball_newind.head()

In [ ]:
baseball_newind.index.is_unique

We can create meaningful indices more easily using a hierarchical index; for now, we will stick with the numeric `id` field as our index.

### Manipulating indices

**Reindexing** allows users to manipulate the data labels in a DataFrame. It forces a DataFrame to conform to the new index, and optionally, fill in missing data if requested.

A simple use of `reindex` is to alter the order of the rows:

In [ ]:
baseball.reindex(baseball.index[::-1]).head()

Notice that the `id` index is not sequential. Say we wanted to populate the table with every `id` value. We could specify and index that is a sequence from the first to the last `id` numbers in the database, and Pandas would fill in the missing data with `NaN` values:

In [ ]:
id_range = range(baseball.index.values.min(), baseball.index.values.max())
baseball.reindex(id_range).head()

Missing values can be filled as desired, either with selected values, or by rule:

In [ ]:
baseball.reindex(id_range, method='ffill', columns=['player','year']).head()

In [ ]:
baseball.reindex(id_range, fill_value='charliebrown', columns=['player']).head()

Keep in mind that `reindex` does not work if we pass a non-unique index series.

We can remove rows or columns via the `drop` method:

In [ ]:
baseball.shape

In [ ]:
baseball.drop([89525, 89526])

In [ ]:
baseball.drop(['ibb','hbp'], axis=1)

## Indexing and Selection

Indexing works analogously to indexing in NumPy arrays, except we can use the labels in the `Index` object to extract values in addition to arrays of integers.

In [ ]:
# Sample Series object
hits = baseball_newind.h
hits

In [ ]:
# Numpy-style indexing
hits[:3]

In [ ]:
# Indexing by label
hits[['womacto01CHN2006','schilcu01BOS2006']]

We can also slice with data labels, since they have an intrinsic order within the Index:

In [ ]:
hits['womacto01CHN2006':'gonzalu01ARI2006']

In [ ]:
hits['womacto01CHN2006':'gonzalu01ARI2006'] = 5
hits

In a `DataFrame` we can slice along either or both axes:

In [ ]:
baseball_newind[['h','ab']]

In [ ]:
baseball_newind[baseball_newind.ab>500]

For a more concise (and readable) syntax, we can use the new `query` method to perform selection on a `DataFrame`. Instead of having to type the fully-specified column, we can simply pass a string that describes what to select. The query above is then simply:

In [ ]:
baseball_newind.query('ab > 500')

The `DataFrame.index` and `DataFrame.columns` are placed in the query namespace by default. If you want to refer to a variable in the current namespace, you can prefix the variable with `@`:

In [ ]:
min_ab = 450

In [ ]:
baseball_newind.query('ab > @min_ab')

The indexing field `loc` allows us to select subsets of rows and columns in an intuitive way:

In [ ]:
baseball_newind.loc['gonzalu01ARI2006', ['h','X2b', 'X3b', 'hr']]

In [ ]:
baseball_newind.loc[:'myersmi01NYA2006', 'hr']

In addition to using `loc` to select rows and columns by **label**, pandas also allows indexing by **position** using the `iloc` attribute.

So, we can query rows and columns by absolute position, rather than by name:

In [ ]:
baseball_newind.iloc[:5, 5:8]

### Exercise

You can use the `isin` method query a DataFrame based upon a list of values as follows: 

    data['phylum'].isin(['Firmacutes', 'Bacteroidetes'])

Use `isin` to find all players that played for the Los Angeles Dodgers (LAN) or the San Francisco Giants (SFN). How many records contain these values?

In [ ]:
# Write your answer here

## Operations

`DataFrame` and `Series` objects allow for several operations to take place either on a single object, or between two or more objects.

For example, we can perform arithmetic on the elements of two objects, such as combining baseball statistics across years. First, let's (artificially) construct two Series, consisting of home runs hit in years 2006 and 2007, respectively:

In [ ]:
hr2006 = baseball.loc[baseball.year==2006, 'hr']
hr2006.index = baseball.player[baseball.year==2006]

hr2007 = baseball.loc[baseball.year==2007, 'hr']
hr2007.index = baseball.player[baseball.year==2007]

In [ ]:
hr2007

Now, let's add them together, in hopes of getting 2-year home run totals:

In [ ]:
hr_total = hr2006 + hr2007
hr_total

Pandas' data alignment places `NaN` values for labels that do not overlap in the two Series. In fact, there are only 6 players that occur in both years.

In [ ]:
hr_total[hr_total.notnull()]

While we do want the operation to honor the data labels in this way, we probably do not want the missing values to be filled with `NaN`. We can use the `add` method to calculate player home run totals by using the `fill_value` argument to insert a zero for home runs where labels do not overlap:

In [ ]:
hr2007.add(hr2006, fill_value=0)

Operations can also be **broadcast** between rows or columns.

For example, if we subtract the maximum number of home runs hit from the `hr` column, we get how many fewer than the maximum were hit by each player:

In [ ]:
baseball.hr - baseball.hr.max()

Or, looking at things row-wise, we can see how a particular player compares with the rest of the group with respect to important statistics

In [ ]:
baseball.loc[89521, "player"]

In [ ]:
stats = baseball[['h','X2b', 'X3b', 'hr']]
diff = stats - stats.loc[89521]
diff[:10]

We can also apply functions to each column or row of a `DataFrame`

In [ ]:
stats.apply(np.median)

In [ ]:
def range_calc(x):
    return x.max() - x.min()

In [ ]:
stat_range = lambda x: x.max() - x.min()
stats.apply(stat_range)

Lets use apply to calculate a meaningful baseball statistics, slugging percentage:

$$SLG = \frac{1B + (2 \times 2B) + (3 \times 3B) + (4 \times HR)}{AB}$$

And just for fun, we will format the resulting estimate.

In [ ]:
def slugging(x): 
    bases = x['h']-x['X2b']-x['X3b']-x['hr'] + 2*x['X2b'] + 3*x['X3b'] + 4*x['hr']
    ab = x['ab']+1e-6
    
    return bases/ab

baseball.apply(slugging, axis=1).round(3)

## Sorting and Ranking

Pandas objects include methods for re-ordering data.

In [ ]:
baseball_newind.sort_index().head()

In [ ]:
baseball_newind.sort_index(ascending=False).head()

Try sorting the **columns** instead of the rows, in ascending order:

In [ ]:
baseball_newind.sort_index(axis=1).head()

We can also use `sort_values` to sort a `Series` by value, rather than by label.

In [ ]:
baseball.hr.sort_values()

For a `DataFrame`, we can sort according to the values of one or more columns using the `by` argument of `sort_values`:

In [ ]:
baseball[['player','sb','cs']].sort_values(ascending=[False,True], 
                                           by=['sb', 'cs']).head(10)

**Ranking** does not re-arrange data, but instead returns an index that ranks each value relative to others in the Series.

In [ ]:
baseball.hr.rank()

Ties are assigned the mean value of the tied ranks, which may result in decimal values.

In [ ]:
pd.Series([100,100]).rank()

Alternatively, you can break ties via one of several methods, such as by the order in which they occur in the dataset:

In [ ]:
baseball.hr.rank(method='first')

Calling the `DataFrame`'s `rank` method results in the ranks of all columns:

In [ ]:
baseball.rank(ascending=False).head()

In [ ]:
baseball[['r','h','hr']].rank(ascending=False).head()

### Exercise

Calculate **on base percentage** for each player, and return the ordered series of estimates.

$$OBP = \frac{H + BB + HBP}{AB + BB + HBP + SF}$$

In [ ]:
# Write your answer here

## Hierarchical indexing

In the baseball example, I was forced to combine 3 fields to obtain a unique index that was not simply an integer value. A more elegant way to have done this would be to create a hierarchical index from the three fields.

In [ ]:
baseball_h = baseball.set_index(['year', 'team', 'player'])
baseball_h.head(10)

This index is a `MultiIndex` object that consists of a sequence of tuples, the elements of which is some combination of the three columns used to create the index. Where there are multiple repeated values, Pandas does not print the repeats, making it easy to identify groups of values.

In [ ]:
baseball_h.index[:10]

In [ ]:
baseball_h.index.is_unique

Try using this hierarchical index to retrieve Julio Franco (`francju01`), who played for the Atlanta Braves (`ATL`) in 2007:

In [ ]:
baseball_h.loc[(2007, 'ATL', 'francju01')]

Recall earlier we imported some microbiome data using two index columns. This created a 2-level hierarchical index:

In [ ]:
mb = pd.read_csv("../data/microbiome.csv", index_col=['Taxon','Patient'])

In [ ]:
mb.head(10)

With a hierachical index, we can select subsets of the data based on a *partial* index:

In [ ]:
mb.loc['Proteobacteria']

Hierarchical indices can be created on either or both axes. Here is a trivial example:

In [ ]:
frame = pd.DataFrame(np.arange(12).reshape(( 4, 3)), 
                  index =[['a', 'a', 'b', 'b'], [1, 2, 1, 2]], 
                  columns =[['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']])

frame

If you want to get fancy, both the row and column indices themselves can be given names:

In [ ]:
frame.index.names = ['key1', 'key2']
frame.columns.names = ['state', 'color']
frame

With this, we can do all sorts of custom indexing:

In [ ]:
frame.loc['a', 'Ohio']

Try retrieving the value corresponding to `b2` in `Colorado`:

In [ ]:
# Write your answer here

Additionally, the order of the set of indices in a hierarchical `MultiIndex` can be changed by swapping them pairwise:

In [ ]:
mb.swaplevel('Patient', 'Taxon').head()

Data can also be sorted by any index level, using `sortlevel`:

In [ ]:
mb.sortlevel('Patient', ascending=False).head()

## Missing data

The occurence of missing data is so prevalent that it pays to use tools like Pandas, which seamlessly integrates missing data handling so that it can be dealt with easily, and in the manner required by the analysis at hand.

Missing data are represented in `Series` and `DataFrame` objects by the `NaN` floating point value. However, `None` is also treated as missing, since it is commonly used as such in other contexts (*e.g.* NumPy).

In [ ]:
foo = pd.Series([np.nan, -3, None, 'foobar'])
foo

In [ ]:
foo.isnull()

Missing values may be dropped or indexed out:

In [ ]:
bacteria2

In [ ]:
bacteria2.dropna()

In [ ]:
bacteria2.isnull()

In [ ]:
bacteria2[bacteria2.notnull()]

By default, `dropna` drops entire rows in which one or more values are missing.

In [ ]:
data.dropna()

This can be overridden by passing the `how='all'` argument, which only drops a row when every field is a missing value.

In [ ]:
data.dropna(how='all')

This can be customized further by specifying how many values need to be present before a row is dropped via the `thresh` argument.

In [ ]:
data.loc[7, 'year'] = np.nan
data

In [ ]:
data.dropna(thresh=4)

This is typically used in time series applications, where there are repeated measurements that are incomplete for some subjects.

### Exercise

Try using the `axis` argument to drop columns with missing values:

In [ ]:
# Write your answer here

Rather than omitting missing data from an analysis, in some cases it may be suitable to fill the missing value in, either with a default value (such as zero) or a value that is either imputed or carried forward/backward from similar data points. We can do this programmatically in Pandas with the `fillna` argument.

In [ ]:
bacteria2.fillna(0)

In [ ]:
data.fillna({'year': 2013, 'treatment':2})

Notice that `fillna` by default returns a new object with the desired filling behavior, rather than changing the `Series` or  `DataFrame` in place (**in general, we like to do this, by the way!**).

We can alter values in-place using `inplace=True`.

In [ ]:
data.year.fillna(2013, inplace=True)
data

Missing values can also be interpolated, using any one of a variety of methods:

In [ ]:
bacteria2.fillna(method='bfill')

## Data summarization

We often wish to summarize data in `Series` or `DataFrame` objects, so that they can more easily be understood or compared with similar data. The NumPy package contains several functions that are useful here, but several summarization or reduction methods are built into Pandas data structures.

In [ ]:
baseball.sum()

Clearly, `sum` is more meaningful for some columns than others. For methods like `mean` for which application to string variables is not just meaningless, but impossible, these columns are automatically exculded:

In [ ]:
baseball.mean()

The important difference between NumPy's functions and Pandas' methods is that the latter have built-in support for handling missing data.

In [ ]:
bacteria2

In [ ]:
bacteria2.mean()

Sometimes we may not want to ignore missing values, and allow the `nan` to propagate.

In [ ]:
bacteria2.mean(skipna=False)

Passing `axis=1` will summarize over rows instead of columns, which only makes sense in certain situations.

In [ ]:
extra_bases = baseball[['X2b','X3b','hr']].sum(axis=1)
extra_bases.sort_values(ascending=False)

A useful summarization that gives a quick snapshot of multiple statistics for a `Series` or `DataFrame` is `describe`:

In [ ]:
baseball.describe()

`describe` can detect non-numeric data and sometimes yield useful information about it.

In [ ]:
baseball.player.describe()

We can also calculate summary statistics *across* multiple columns, for example, correlation and covariance.

$$cov(x,y) = \sum_i (x_i - \bar{x})(y_i - \bar{y})$$

In [ ]:
baseball.hr.cov(baseball.X2b)

$$corr(x,y) = \frac{cov(x,y)}{(n-1)s_x s_y} = \frac{\sum_i (x_i - \bar{x})(y_i - \bar{y})}{\sqrt{\sum_i (x_i - \bar{x})^2 \sum_i (y_i - \bar{y})^2}}$$

In [ ]:
baseball.hr.corr(baseball.X2b)

In [ ]:
baseball.ab.corr(baseball.h)

Try running `corr` on the entire `baseball` DataFrame to see what is returned:

In [ ]:
# Write answer here

If we have a `DataFrame` with a hierarchical index (or indices), summary statistics can be applied with respect to any of the index levels:

In [ ]:
mb.head()

In [ ]:
mb.sum(level='Taxon')

## Writing Data to Files

As well as being able to read several data input formats, Pandas can also export data to a variety of storage formats. We will bring your attention to just a couple of these.

In [ ]:
mb.to_csv("mb.csv")

The `to_csv` method writes a `DataFrame` to a comma-separated values (csv) file. You can specify custom delimiters (via `sep` argument), how missing values are written (via `na_rep` argument), whether the index is writen (via `index` argument), whether the header is included (via `header` argument), among other options.

An efficient way of storing data to disk is in binary format. Pandas supports this using Python’s built-in pickle serialization.

In [ ]:
baseball.to_pickle("baseball_pickle")

The complement to `to_pickle` is the `read_pickle` function, which restores the pickle to a `DataFrame` or `Series`:

In [ ]:
pd.read_pickle("baseball_pickle")

As Wes warns in his book, it is recommended that binary storage of data via pickle only be used as a temporary storage format, in situations where speed is relevant. This is because there is no guarantee that the pickle format will not change with future versions of Python.

### Advanced Exercise: Compiling Ebola Data

The `data/ebola` folder contains summarized reports of Ebola cases from three countries during the recent outbreak of the disease in West Africa. For each country, there are daily reports that contain various information about the outbreak in several cities in each country.

From these data files, use pandas to import them and create a single data frame that includes the daily totals of new cases and deaths for each country.

## Ejercicios: 